# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [ ]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-lattytytyty') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [ ]:
# กำหนดพื้นที่สนใจ นครปฐม
geometry = ee.Geometry.Point([100.0615, 13.8199])

# เรียกภาพ L9 ตัวอย่างแล้วดึง RGB และ Pan ออกมา
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2023-01-01', '2023-03-01')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [ ]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [ ]:
# สร้างแผนที่
Map = geemap.Map(center=[100.0615, 13.8199], zoom=10)

# แสดง เลเยอร์ ก่อนและหลังโดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


Map(center=[100.0615, 13.8199], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

ตอบ จากการใช้Landsat-9 ภาพmultispectral จะมีความละเอียดอยู่ที่ 30 เมตร ในขณะที่ panchromatic band จะมีความละเอียดสูงกว่า คือ 15 เมตร ซึ่งเมื่อนำรายละเอียดของสองอย่างนี้มาผสานกันจะทำให้ภาพมีรายละเอียดชัดมากขึ้น เหมาะแก่การแปลภาพ การวิเคราะห์การใช้ที่ดิน และการศึกษาการเปลี่ยนแปลงพื้นที่


2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร

ตอบ ขั้นตอนสำคัญของการทำ Pan-sharpening มีดังนี้
1. เลือกภาพ Multispectral (เช่น BA, B3, B2) เพื่อให้ข้อมูลสี
2. เลือกภาพ Panchromatic (B8) ซึ่งมีความละเอียดเชิงพื้นที่สูงกว่า
3. ปรับค่าสะท้อนแสงของภาพให้อยู่ในช่วงที่เหมาะสม
4. แปลงภาพสีจากระบบ RGB เป็น HSV
5. แทนที่แบนด์ Value (V) ด้วยภาพ Panchromatic เพื่อเพิ่มรายละเอียดเชิงพื้นที่
6. แปลงกลับจาก HSV เป็น RGB เพื่อให้ได้ภาพสีที่คมชัดขึ้น
กระบวนการนี้ช่วยผสานจุดเด่นของทั้งสองข้อมูล คือ สีจาก Multispectral และความคมชัดจาก Panchromatic ทำให้ได้ภาพสีที่มีรายละเอียดสูง เหมาะสำหรับการวิเคราะห์เชิงพื้นที่มากยิ่งขึ้น